# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

# Names

- Susmitha Kalidindi 
- Alyssa Ashmore 
- Shardul Saiya 
- Siyi Wang


# Group Members IDs

- A13167684
- A12833150
- A13964199
- A53233349

# Research Question

- Question 1: Want to study how strong the correlation is between words used in product reviews with the ratings given for that product
- Question 2: Want to study the relationship between specific language that users use in reviews, the value of the  ratings they give and whether they are “fake” or incentivized reviews

Edit: Mapping the keywords in Amazon review text against the review ratings to see if words can be used to predict ratings.

## Background and Prior Work

Background:

References:
- 1) https://reviewmeta.com/
- 2) 

# Hypothesis


# Dataset(s)

Dataset Name: Consumer Reviews of Amazon Products
Link to the dataset: https://www.kaggle.com/datafiniti/consumer-reviews-of-amazon-products#Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv
Number of observations: 34,000 reviews

This dataset contains a list of over 34,000 reviews of Amazon products like the Kindle, Fire TV, including measures on the 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [ ]:
# setup for using the nltk library
import nltk
#nltk.download() # NEED TO DOWNLOAD THIS TO USE NLTK 
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [ ]:
## Import Data from CSV file here, SEE Project proposal for link to csv file
import pandas as pd 
dfraw = pd.read_csv("../consumer-reviews-of-amazon-products/1429_1.csv", engine = 'python')
df2 = pd.read_csv("../consumer-reviews-of-amazon-products/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv", engine = 'python')


# Data Cleaning

Describe your data cleaning steps here.

In [ ]:
# Get rid of most unwanted rows, only keep 
# Name, Category, Reviews.DoRecommend, Review.numHelpful, Reviews.ratings, Review.text, Review.title 

df1 = dfraw.drop(['asins', 'brand', 'keys', 'manufacturer', 'reviews.sourceURLs', 'reviews.userCity', 
                  'reviews.userProvince', 'reviews.username', 'reviews.dateAdded', 'reviews.dateSeen', 
                  'reviews.didPurchase' , 'reviews.id', 'id' ], axis=1)

#get rid of rewiews without ratings
df1 = df1.dropna(subset=['reviews.rating', 'reviews.text', 'reviews.text'])

# Now df1 contains:
# name, categories, reviews.date, reviews.doRecommend,
# reviews.numHelpful, reviews.rating, reviews.text, reviews.title

In [ ]:
#helper function to filter data
def filter_review(review):
    # Convert to string
    string = str(review)
    
    # Data Filtering
    string = string.replace('.', ' ')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    # string = string.replace('\'', ' ')
    string = string.replace('\"', ' ')
    string = string.replace('/', ' ')
    string = string.replace('(', ' ')
    string = string.replace(')', ' ')
    string = string.replace('!', ' ')
    string = string.replace('?', ' ')
    string = string.replace('=', ' ')
    string = string.replace('+', ' ')
    string = string.replace('-', ' ')
    string = string.replace('�', '')
    string = string.replace('☺️', ' ')
    string = string.replace('&', ' ')
    string = string.replace('$', ' ')
    string = string.replace('*', ' ')
    string = string.replace('%', ' ')

    
    return string

In [ ]:
# dictionary to count all unique words in customer reviews
# contains word : frequency
words_dict = {}

# Helper function to count words in a string 
# This function directly modifies the words_dict declared above
def word_frequency_counter(review):
    
    string = filter_review(review)

    # Convert to lower case and split
    my_string = string.lower().split()
    for item in my_string:
        if item in words_dict:
            words_dict[item] += 1
        else:
            words_dict[item] = 1

In [ ]:
# dictionary to count raw rating value of each word
rating_dict = {}

#Helper function to add up raw review values for each word and 
def rating_counter(review, rating):
    string = filter_review(review)
    rating = int(rating)
    
    # Convert to lower case and split
    my_string = string.lower().split()
    for item in my_string:
        if item in rating_dict:
            rating_dict[item] = rating_dict[item] + rating
        else:
            rating_dict[item] = rating
    

In [ ]:
for review in df1['reviews.text']:
    word_frequency_counter(review)
print(words_dict)
for index, row in df1.iterrows():
    rating_counter(row['reviews.text'], row['reviews.rating'])

In [ ]:
# TODO
# Visualize word frequency's for top words with a histogram or some other graph
sorted_words_list = list(sorted(words_dict.items(), key=lambda item: item[1], reverse=True))
words_list_20 = sorted_words_list[0:20] 
df_words = pd.DataFrame(words_list_20, columns=['word', 'frequency'])
df_words.plot(kind='bar', x='word')

In [ ]:
# TODO: (Susmitha) visualize the graph removing neutral words like "the", "it" to see if popular words are mainly positive or negative
# using NLTK 

sid = SentimentIntensityAnalyzer()
#rint(ss)
import copy
sorted_words_list = list(sorted(words_dict.items(), key=lambda item: item[1], reverse=True))
sorted_words_list_nonneutral = copy.deepcopy(sorted_words_list)
for item in sorted_words_list:
    ss = sid.polarity_scores(item[0])
    # if word is neutral, then don't include it in this graph
    # we only care about positive/negative words
    
    if ss['neu'] == 1.0:
        sorted_words_list_nonneutral.remove(item)    

In [ ]:
# Visualize word frequency's for top words with a histogram or some other graph
words_list_20 = sorted_words_list_nonneutral[0:20] 
df_words = pd.DataFrame(words_list_20, columns=['word', 'frequency'])
df_words.plot(kind='bar', x='word')

In [ ]:
# Now we have two dictionaries
# word_dict contains words associated with their frequency in all the ratings
# rating_dict_raw cotains words associated with their raw rating values, summed up


# Now we want to divide the values in rating_dict by the frequency values in word_dict
# Thats what this helper function does, and saves it in rating_dict
def ratings_avg(word_dict, rating_dict):
    for item in word_dict:
        if item in rating_dict:
            rating_dict[item] = rating_dict[item]/word_dict[item]
        else:
            print('This shouldnt happen !!')
    

In [ ]:
ratings_avg(words_dict, rating_dict)
print(rating_dict)

In [ ]:
# TODO
# Visualize these average rating values with words


# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [ ]:
# Now rating_dict has what we need to go ahead


# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*